In [3]:
import requests
import json
import sys
import pandas as pd
from tabulate import tabulate
from flask import Flask, render_template
from bs4 import BeautifulSoup

In [107]:
#apis documentation: https://docs.scrapeak.com/zillow-scraper/overview

api_key = "0c141748-c9da-44f6-b91c-bd08e529f3a5"
api_url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"
listings_url = "https://www.zillow.com/homes/for_rent/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-122.17040010111008%2C%22east%22%3A-122.0699781955925%2C%22south%22%3A37.39634394555346%2C%22north%22%3A37.44542356875894%7D%2C%22mapZoom%22%3A14%2C%22usersSearchTerm%22%3A%223444%20Ashton%20Ct%20Palo%20Alto%2C%20CA%2094306%22%2C%22customRegionId%22%3A%221e42866394X1-CR8uzkls0vwlfr_13lqr1%22%2C%22filterState%22%3A%7B%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sche%22%3A%7B%22value%22%3Afalse%7D%2C%22schm%22%3A%7B%22value%22%3Afalse%7D%2C%22schh%22%3A%7B%22value%22%3Afalse%7D%2C%22schp%22%3A%7B%22value%22%3Afalse%7D%2C%22schr%22%3A%7B%22value%22%3Afalse%7D%2C%22schc%22%3A%7B%22value%22%3Afalse%7D%2C%22schu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"
params = {"api_key": api_key, "url": listings_url}

response = requests.get(api_url, params=params)
response_json = response.json()

if not response_json: # If creating a web user interface have this return an error code using flask
    print("Failed to get listings")
    sys.exit()

# get the zpids (unique property ids) for each property returned in the listings search
data = response_json["data"]
zpids={}
for i in data["cat1"]["searchResults"]["listResults"]:
    zpids[i["zpid"]] = i["detailUrl"]

# find the price history for each property returned
api_url = "https://app.scrapeak.com/v1/scrapers/zillow/property"
pricehistory=[]
for zpid in zpids:
    params = {"api_key": api_key, "zpid": zpid}
    pricehistory.append({
        "zpid": zpid,
        "detailsurl": zpids[str(zpid)],
        "pricehistory": requests.get(api_url, params=params).json()["data"]["priceHistory"]
    })


In [163]:
print(data["cat1"]["searchResults"]["listResults"][0].keys())
print(len(zpids))
#for i in zpids:
#    print(i)
#for i in pricehistory:
#    print(i["zpid"])

dict_keys(['zpid', 'id', 'rawHomeStatusCd', 'marketingStatusSimplifiedCd', 'providerListingId', 'imgSrc', 'hasImage', 'detailUrl', 'statusType', 'statusText', 'countryCurrency', 'price', 'unformattedPrice', 'address', 'addressStreet', 'addressCity', 'addressState', 'addressZipcode', 'isUndisclosedAddress', 'beds', 'baths', 'area', 'latLong', 'isZillowOwned', 'variableData', 'hdpData', 'isSaved', 'isUserClaimingOwner', 'isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate', 'shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec', 'hasAdditionalAttributions', 'isFeaturedListing', 'isShowcaseListing', 'availabilityDate', 'list', 'relaxed', 'carouselPhotos', 'rooms'])
11


In [162]:
#print(pricehistory[0]["pricehistory"][0].keys())
# dict_keys(['date', 'time', 'price', 'pricePerSquareFoot', 'priceChangeRate', 'event', 'source', 'buyerAgent', 'sellerAgent', 'showCountyLink', 'postingIsRental', 'attributeSource'])
print(pricehistory[0]["pricehistory"][0])

#data["cat1"]["searchResults"]["listResults"][0]
# I want detailUrl, price, address, beds, baths, area

{'date': '2024-05-01', 'time': 1714521600000, 'price': 6595, 'pricePerSquareFoot': 5, 'priceChangeRate': 0.10840336134453782, 'event': 'Listed for rent', 'source': 'Zillow Rentals', 'buyerAgent': None, 'sellerAgent': None, 'showCountyLink': False, 'postingIsRental': True, 'attributeSource': {'infoString1': None, 'infoString2': 'Zillow Rentals', 'infoString3': None}}


In [1]:
# Get a summary of properties returned
summary_table = []
for i in data["cat1"]["searchResults"]["listResults"]:
    try:
        area = i["area"]
    except KeyError:
        area = "null"
    row = [
        i["price"],
        area,
        i["beds"],
        i["baths"],
        i["address"],
        i["detailUrl"]
    ]
    summary_table.append(row)
summary_table()
print(tabulate(summary_table, headers=["Price", "Area", "# Beds", "# Baths", "Address", "Zillow Link"]))

NameError: name 'data' is not defined

In [119]:
for property in pricehistory:
    print(property["detailsurl"])
    table = []
    for history in property["pricehistory"]:
        row = [
            history["date"],
            history["event"],
            history["price"],
            history["priceChangeRate"]
        ]
        table.append(row)
    print(tabulate(table, headers=["Date", "Event", "Price", "Price Change"]))


https://www.zillow.com/homedetails/Palo-Alto-CA-94306/19502746_zpid/
Date        Event              Price    Price Change
----------  ---------------  -------  --------------
2024-05-01  Listed for rent     6595        0.108403
2023-05-13  Listing removed                 0
2023-04-29  Listed for rent     5950        0
https://www.zillow.com/homedetails/747-Holly-Oak-Dr-Palo-Alto-CA-94303/19500649_zpid/
Date        Event              Price    Price Change
----------  ---------------  -------  --------------
2024-05-04  Listed for rent     8495               0
https://www.zillow.com/homedetails/3737-Middlefield-Rd-Palo-Alto-CA-94303/19500367_zpid/
Date        Event              Price    Price Change
----------  ---------------  -------  --------------
2024-04-27  Listed for rent    13000               0
2003-07-14  Sold              770000               0
https://www.zillow.com/homedetails/3780-Wright-Pl-Palo-Alto-CA-94306/19503395_zpid/
Date        Event              Price    Price Chan

{'zpid': '19502746',
 'id': '19502746',
 'rawHomeStatusCd': 'ForRent',
 'marketingStatusSimplifiedCd': 'For Rent',
 'providerListingId': '3azf0tg47bqd3',
 'imgSrc': 'https://photos.zillowstatic.com/fp/38d02fd0bebdaac3ccdc353a8a4e5d62-p_e.jpg',
 'hasImage': True,
 'detailUrl': 'https://www.zillow.com/homedetails/Palo-Alto-CA-94306/19502746_zpid/',
 'statusType': 'FOR_RENT',
 'statusText': 'House for rent',
 'countryCurrency': '$',
 'price': '$6,595/mo',
 'unformattedPrice': 6595,
 'address': '(undisclosed Address), Palo Alto, CA 94306',
 'addressStreet': '(undisclosed Address)',
 'addressCity': 'Palo Alto',
 'addressState': 'CA',
 'addressZipcode': '94306',
 'isUndisclosedAddress': True,
 'beds': 3,
 'baths': 2.0,
 'area': 1400,
 'latLong': {},
 'isZillowOwned': False,
 'variableData': {'type': 'TIME_ON_INFO',
  'text': '5 days ago',
  'data': {'isRead': None, 'isFresh': False}},
 'hdpData': {'homeInfo': {'zpid': 19502746,
   'streetAddress': '(undisclosed Address)',
   'zipcode': '9430

In [161]:
#print(data["cat1"]["searchResults"]["listResults"][4])
print(data["cat1"]["searchResults"]["listResults"][0].keys())
#print(data["cat1"]["searchResults"]["listResults"][0]["area"])
for i in data["cat1"]["searchResults"]["listResults"]:
    print(i["list"])
    print(i["availabilityDate"])


dict_keys(['zpid', 'id', 'rawHomeStatusCd', 'marketingStatusSimplifiedCd', 'providerListingId', 'imgSrc', 'hasImage', 'detailUrl', 'statusType', 'statusText', 'countryCurrency', 'price', 'unformattedPrice', 'address', 'addressStreet', 'addressCity', 'addressState', 'addressZipcode', 'isUndisclosedAddress', 'beds', 'baths', 'area', 'latLong', 'isZillowOwned', 'variableData', 'hdpData', 'isSaved', 'isUserClaimingOwner', 'isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate', 'shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec', 'hasAdditionalAttributions', 'isFeaturedListing', 'isShowcaseListing', 'availabilityDate', 'list', 'relaxed', 'carouselPhotos', 'rooms'])
True
2024-07-01 00:00:00
True
2024-05-18 00:00:00
True
2024-06-15 00:00:00
True


KeyError: 'availabilityDate'

In [7]:
listings_url = "https://www.zillow.com/homes/for_rent/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-122.17040010111008%2C%22east%22%3A-122.0699781955925%2C%22south%22%3A37.39634394555346%2C%22north%22%3A37.44542356875894%7D%2C%22mapZoom%22%3A14%2C%22usersSearchTerm%22%3A%223444%20Ashton%20Ct%20Palo%20Alto%2C%20CA%2094306%22%2C%22customRegionId%22%3A%221e42866394X1-CR8uzkls0vwlfr_13lqr1%22%2C%22filterState%22%3A%7B%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sche%22%3A%7B%22value%22%3Afalse%7D%2C%22schm%22%3A%7B%22value%22%3Afalse%7D%2C%22schh%22%3A%7B%22value%22%3Afalse%7D%2C%22schp%22%3A%7B%22value%22%3Afalse%7D%2C%22schr%22%3A%7B%22value%22%3Afalse%7D%2C%22schc%22%3A%7B%22value%22%3Afalse%7D%2C%22schu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:55.0) Gecko/20100101 Firefox/55.0"}    
r = requests.get("https://www.zillow.com/homes/recently_sold/Culver-City-CA/house,condo,apartment_duplex,townhouse_type/20432063_zpid/51617_rid/12m_days/globalrelevanceex_sort/34.048605,-118.340178,33.963223,-118.47785_rect/12_zm/", headers=headers)
print(r)

<Response [403]>
